In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Cleaned_Data')

In [3]:
X=df.drop(columns='AQI')
y=df["AQI"]

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X, y, train_size=0.8, random_state=10)

In [6]:
from sklearn.decomposition import PCA
pc = PCA(n_components=0.95)
x_train = pc.fit_transform(x_train)
x_test = pc.transform(x_test)

In [7]:
pc.explained_variance_ratio_

array([0.40647262, 0.14163057, 0.13359959, 0.10952585, 0.10803623,
       0.0461153 , 0.02457858])

In [8]:
#modeling
from sklearn.linear_model import Lasso
mod = Lasso()
mod.fit(x_train,y_train)
print('intercept :',mod.intercept_)
print('Coefficient :',mod.coef_)


#predection
x_train_pred = mod.predict(x_train)
x_test_pred = mod.predict(x_test)

#evolution
print('Train R2 :',mod.score(x_train,y_train))
print('Test R2 :',mod.score(x_test,y_test))

#CV Score
from sklearn.model_selection import cross_val_score
cv = cross_val_score(mod,x_train,y_train,cv=5,scoring='r2')
print('CV Score :',cv.mean())

intercept : 41.256055342202764
Coefficient : [ 9.07483267  7.47837436 -3.66877983  0.          0.44186048  0.
  0.        ]
Train R2 : 0.7372513525766713
Test R2 : 0.7364393099483733
CV Score : 0.7371722204815573


**Checking best alpha value**

In [10]:
from sklearn.model_selection import GridSearchCV
est = Lasso()
listt = {'alpha':list(range(1,100))}
mod = GridSearchCV(est,listt,cv=5,scoring='neg_mean_squared_error')
mod.fit(x_train,y_train)
mod.best_params_

{'alpha': 1}

**Remodeling**

In [12]:
df1 = pd.DataFrame(x_train)
df1

,0,1,2,3,4,5,6
0,0.662371,-1.501612,-0.602141,-1.544715,0.615640,0.806596,0.730567
1,-2.799253,1.359548,1.220022,-0.816934,-0.160953,-0.026566,-1.102495
2,0.334586,-1.535491,-0.429115,1.510595,-1.685778,0.530323,-0.291950
3,-3.648238,0.530249,-1.013583,0.243336,1.487058,-1.561101,-0.696102
4,-1.742039,0.941067,1.838339,-1.014250,-1.312504,0.431391,-0.038513
...,...,...,...,...,...,...,...
42151,0.058911,-1.300180,-0.769973,1.525696,-1.534880,0.496310,0.065825
42152,-1.709457,1.451957,1.341619,-0.923835,-0.750256,0.524305,-0.053250
42153,-0.738179,-0.718353,-0.878636,1.610613,-1.134875,0.719723,-0.006174
42154,0.577816,-0.825758,-0.814424,-1.397640,1.173584,1.364405,0.032147


In [13]:
X=df1[[0,1,2,4]]
y=df["AQI"]

In [14]:
#modeling
from sklearn.linear_model import Lasso
mod = Lasso(alpha=1)
mod.fit(x_train,y_train)
print('intercept :',mod.intercept_)
print('intercept :',mod.coef_)


#predection
x_train_pred = mod.predict(x_train)
x_test_pred = mod.predict(x_test)

#evolution
print('Train R2 :',mod.score(x_train,y_train))
print('Test R2 :',mod.score(x_test,y_test))

#CV Score
from sklearn.model_selection import cross_val_score
cv = cross_val_score(mod,x_train,y_train,cv=5,scoring='r2')
print('CV Score :',cv.mean())

intercept : 41.256055342202764
intercept : [ 9.07483267  7.47837436 -3.66877983  0.          0.44186048  0.
  0.        ]
Train R2 : 0.7372513525766713
Test R2 : 0.7364393099483733
CV Score : 0.7371722204815573


In [15]:
#Checking RMSE
from sklearn.metrics import mean_squared_error
Train_MSE=mean_squared_error(y_train,x_train_pred)
Test_MSE=mean_squared_error(y_test,x_test_pred)
print('Train RMSE :',Train_MSE**0.5)
print('Test RMSE :',Test_MSE**0.5)
cv = cross_val_score(mod,x_train,y_train,cv=5,scoring='neg_mean_squared_error')
cv = abs(cv.mean())
print ('CV Score :',cv**0.5)

Train RMSE : 13.565447061400716
Test RMSE : 13.993855637990471
CV Score : 13.566639569442831
